In [1]:
import wandb
wandb.login(key="5fb34431b405eb21dc0f263e5b3cf2c15fdc7471")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ma24m004 (ma24m004-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [7]:
# !pip install yattag


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from torch.nn.utils.rnn import pad_sequence  # <-- ADD THIS LINE


In [4]:
class Attn_Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, n_layers, cell_type, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = getattr(nn, cell_type)(emb_dim, hidden_dim, 
                                        n_layers, dropout=dropout, 
                                        batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)
        if isinstance(hidden, tuple):  # LSTM
            return outputs, hidden[0].contiguous(), hidden[1].contiguous()
        else:  # GRU/RNN
            return outputs, hidden.contiguous(), None



In [5]:
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.W = nn.Linear(hidden_dim, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)

    def forward(self, decoder_hidden, encoder_outputs):
        # decoder_hidden: (batch_size, hidden_dim)
        # encoder_outputs: (batch_size, seq_len, hidden_dim)
        
        # Repeat decoder hidden for each encoder time step
        decoder_hidden = decoder_hidden.unsqueeze(1).expand_as(encoder_outputs)
        
        # Calculate attention energies
        energy = torch.tanh(self.W(decoder_hidden + encoder_outputs))
        attention = self.V(energy).squeeze(2)
        
        return torch.softmax(attention, dim=1)


In [6]:
class Attn_Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, n_layers, cell_type, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.attention = Attention(hidden_dim)
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = getattr(nn, cell_type)(emb_dim + hidden_dim, hidden_dim, 
                                        n_layers, dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell, encoder_outputs):
        input = input.unsqueeze(1)  # (batch_size, 1)
        embedded = self.dropout(self.embedding(input))  # (batch_size, 1, emb_dim)
    
        # Calculate attention weights: (batch_size, src_len)
        attn_weights = self.attention(hidden[-1], encoder_outputs)
    
        # Create context vector: (batch_size, 1, hidden_dim)
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs)
    
        # Concatenate context and embedded: (batch_size, 1, emb_dim + hidden_dim)
        rnn_input = torch.cat([embedded, context], dim=2)
    
        # Pass through RNN
        if cell is not None:
            output, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))
        else:
            output, hidden = self.rnn(rnn_input, hidden)
            cell = None  # To ensure cell is always returned
    
        # Generate prediction
        output = torch.cat([output.squeeze(1), context.squeeze(1)], dim=1)  # (batch_size, hidden_dim * 2)
        prediction = self.fc_out(output)  # (batch_size, output_dim)
    
        return prediction, hidden, cell, attn_weights  # <-- include attn_weights


In [7]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import pandas as pd


# Seq2Seq Model with Beam Search
class Attn_Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, sos_idx, eos_idx):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.sos_idx = sos_idx
        self.eos_idx = eos_idx

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        
        # ✅ Fix here (only 3 returned)
        encoder_outputs, hidden, cell = self.encoder(src)
        
        inputs = trg[:, 0]
        outputs = torch.zeros(batch_size, trg_len, self.decoder.output_dim).to(self.device)
    
        for t in range(1, trg_len):
            # Decoder returns 4, we only care about first 3 during training
            output, hidden, cell, _ = self.decoder(inputs, hidden, cell, encoder_outputs)
            outputs[:, t] = output
            teacher_force = np.random.random() < teacher_forcing_ratio
            inputs = trg[:, t] if teacher_force else output.argmax(1)
            
        return outputs


    def predict(self, src, max_len=30, beam_size=3):
        self.eval()
        batch_size = src.size(0)
        assert batch_size == 1, "Beam search only supports batch_size=1"
    
        with torch.no_grad():
            encoder_outputs, hidden, cell = self.encoder(src)
    
            beams = [(torch.tensor([self.sos_idx], device=self.device), 0.0, hidden, cell, [])]
            completed_sequences = []
    
            for _ in range(max_len):
                new_beams = []
                for seq, score, h, c, attn_list in beams:
                    last_token = seq[-1].unsqueeze(0)
                    if last_token.item() == self.eos_idx:
                        completed_sequences.append((seq, score, attn_list))  # ✅ 3-tuple
                        continue
    
                    output, h_new, c_new, attn_weights = self.decoder(last_token, h, c, encoder_outputs)
                    log_probs = torch.log_softmax(output, dim=1).squeeze(0)
                    topk = torch.topk(log_probs, beam_size)
    
                    for idx, log_prob in zip(topk.indices, topk.values):
                        new_seq = torch.cat([seq, idx.unsqueeze(0)])
                        new_score = score + log_prob.item()
                        new_attn_list = attn_list + [attn_weights.cpu()]
                        new_beams.append((new_seq, new_score, h_new, c_new, new_attn_list))
    
                beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]
    
                if all(seq[-1].item() == self.eos_idx for seq, _, _, _, _ in beams):
                    completed_sequences.extend([
                        (seq, score, attn_list) for seq, score, _, _, attn_list in beams
                    ])
                    break
    
            # ✅ Fix: Ensure 3-tuple fallback
            if not completed_sequences:
                completed_sequences = [
                    (seq, score, attn_list) for seq, score, _, _, attn_list in beams
                ]
    
            best_seq, _, best_attn_list = max(completed_sequences, key=lambda x: x[1])
            return best_seq[1:], best_attn_list  # Remove <SOS> token
    

# Dataset
class DakshinaDataset(Dataset):
    def __init__(self, data, input_vocab, output_vocab):
        self.data = data
        self.input_vocab = input_vocab
        self.output_vocab = output_vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        src = [self.input_vocab.get(c, self.input_vocab['<UNK>']) for c in self.data.iloc[idx, 1]] + [self.input_vocab['<EOS>']]
        tgt = [self.output_vocab['<SOS>']] + [self.output_vocab.get(c, self.output_vocab['<UNK>']) for c in self.data.iloc[idx, 0]] + [self.output_vocab['<EOS>']]
        return torch.tensor(src, dtype=torch.long), torch.tensor(tgt, dtype=torch.long)


# Vocab Creation
def create_vocab(data, column):
    vocab = {'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3}
    for seq in data[column]:
        if pd.notna(seq):
            for char in seq:
                if char not in vocab:
                    vocab[char] = len(vocab)
    return vocab


# Collate Function
def pad_collate(batch):
    src_batch, tgt_batch = zip(*batch)
    src_padded = pad_sequence(src_batch, batch_first=True, padding_value=0)
    tgt_padded = pad_sequence(tgt_batch, batch_first=True, padding_value=0)
    return src_padded, tgt_padded


In [8]:
def Attn_train_model(config=None):
    with wandb.init(config=config):
        config = wandb.config
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Load data
        train_data = pd.read_csv('/kaggle/input/dakshina/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv', sep='\t', header=None, dtype=str).dropna()
        dev_data = pd.read_csv('/kaggle/input/dakshina/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv', sep='\t', header=None, dtype=str).dropna()

        # Create vocabularies
        input_vocab = create_vocab(train_data, 1)  # English (source)
        output_vocab = create_vocab(train_data, 0)  # Hindi (target)

        # Prepare datasets and loaders
        train_dataset = DakshinaDataset(train_data, input_vocab, output_vocab)
        dev_dataset = DakshinaDataset(dev_data, input_vocab, output_vocab)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=pad_collate)
        dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=False, collate_fn=pad_collate)

        # Initialize model
        encoder = Attn_Encoder(len(input_vocab), config.emb_dim, config.hidden_dim, config.num_layers, config.cell_type, config.dropout)
        decoder = Attn_Decoder(len(output_vocab), config.emb_dim, config.hidden_dim, config.num_layers, config.cell_type, config.dropout)
        model = Attn_Seq2Seq(
            encoder, decoder, device,
            sos_idx=output_vocab['<SOS>'],
            eos_idx=output_vocab['<EOS>']
        ).to(device)

        # Loss and optimizer
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss(ignore_index=0)

        best_val_accuracy = 0
        patience_counter = 0

        # Training loop
        for epoch in range(config.epochs):
            model.train()
            train_loss, train_correct, train_total = 0, 0, 0
            for src, tgt in train_loader:
                src, tgt = src.to(device), tgt.to(device)
                optimizer.zero_grad()
                output = model(src, tgt)
                output = output[:, 1:].reshape(-1, output.shape[-1])
                tgt = tgt[:, 1:].reshape(-1)
                loss = criterion(output, tgt)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                preds = output.argmax(dim=1)
                non_pad_mask = tgt != 0
                train_correct += (preds == tgt)[non_pad_mask].sum().item()
                train_total += non_pad_mask.sum().item()

            train_accuracy = train_correct / train_total

            # Validation
            model.eval()
            val_loss, val_correct, val_total = 0, 0, 0
            with torch.no_grad():
                for src, tgt in dev_loader:
                    src, tgt = src.to(device), tgt.to(device)
                    output = model(src, tgt, teacher_forcing_ratio=0)
                    output = output[:, 1:].reshape(-1, output.shape[-1])
                    tgt = tgt[:, 1:].reshape(-1)
                    loss = criterion(output, tgt)
                    val_loss += loss.item()
                    preds = output.argmax(dim=1)
                    non_pad_mask = tgt != 0
                    val_correct += (preds == tgt)[non_pad_mask].sum().item()
                    val_total += non_pad_mask.sum().item()

            val_accuracy = val_correct / val_total

            # Save best model
            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                torch.save(model.state_dict(), '/kaggle/working/Attn_best_model.pt')
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= 4:
                    print("Early stopping triggered.")
                    break

            # Log metrics to wandb
            wandb.log({
                'epoch': epoch,
                'train_loss': train_loss / len(train_loader),
                'val_loss': val_loss / len(dev_loader),
                'train_accuracy': train_accuracy,
                'val_accuracy': val_accuracy
            })

            # Log sample predictions
            src_sample, tgt_sample = next(iter(dev_loader))
            src_sample, tgt_sample = src_sample.to(device), tgt_sample.to(device)
            preds = []
            for i in range(5):
                single_input = src_sample[i].unsqueeze(0)
                pred_seq, _ = model.predict(single_input, beam_size=config.beam_size)  # <-- FIX: unpack properly
                preds.append(pred_seq)

            inv_input_vocab = {v: k for k, v in input_vocab.items()}
            inv_output_vocab = {v: k for k, v in output_vocab.items()}
            table = wandb.Table(columns=["Input", "Target", "Prediction"])
            for i in range(len(preds)):
                input_str = ''.join([inv_input_vocab.get(id.item(), '?') for id in src_sample[i] if id.item() not in [0, input_vocab['<EOS>']]])
                target_str = ''.join([inv_output_vocab.get(id.item(), '?') for id in tgt_sample[i] if id.item() not in [0, output_vocab['<EOS>']]])
                
                pred_tokens = []
                for idx in preds[i]:
                    idx_val = idx.item() if isinstance(idx, torch.Tensor) else idx
                    if idx_val == output_vocab['<EOS>']:
                        break
                    pred_tokens.append(inv_output_vocab.get(idx_val, '?'))
                pred_str = ''.join(pred_tokens)

                table.add_data(input_str, target_str, pred_str)

            wandb.log({"Predictions": table})


In [15]:

# Sweep
sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'emb_dim': {'values': [16, 32, 64, 256]},
        'hidden_dim': {'values': [16, 32, 64, 256]},
        'num_layers': {'values': [1, 2, 3]},
        'cell_type': {'values': ['RNN', 'GRU', 'LSTM']},
        'dropout': {'values': [0.2, 0.3]},
        'epochs': {'values': [10, 5]},
        'beam_size': {'values': [1, 3, 5]}
    }
}

sweep_id = wandb.sweep(sweep_config, project='DL_ASSIGNMENT_3_RNN')
wandb.agent(sweep_id, Attn_train_model, count=10)
wandb.finish()

In [15]:
!fc-list | grep Devanagari

In [16]:
!wget https://noto-website-2.storage.googleapis.com/pkgs/NotoSansDevanagari-hinted.zip
!unzip NotoSansDevanagari-hinted.zip

--2025-05-20 04:50:19--  https://noto-website-2.storage.googleapis.com/pkgs/NotoSansDevanagari-hinted.zip
Resolving noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)... 173.194.210.207, 108.177.12.207, 142.251.107.207, ...
Connecting to noto-website-2.storage.googleapis.com (noto-website-2.storage.googleapis.com)|173.194.210.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6201435 (5.9M) [application/zip]
Saving to: ‘NotoSansDevanagari-hinted.zip’

NotoSansDevanagari- 100%[===================>]   5.91M  --.-KB/s    in 0.08s   

2025-05-20 04:50:20 (75.2 MB/s) - ‘NotoSansDevanagari-hinted.zip’ saved [6201435/6201435]

Archive:  NotoSansDevanagari-hinted.zip
  inflating: LICENSE_OFL.txt         
  inflating: NotoSansDevanagari-Black.ttf  
  inflating: NotoSansDevanagari-Bold.ttf  
  inflating: NotoSansDevanagari-Condensed.ttf  
  inflating: NotoSansDevanagari-CondensedBlack.ttf  
  inflating: NotoSansDevanagari-CondensedBold.ttf  


In [17]:
import os
import matplotlib.pyplot as plt
from matplotlib import font_manager, rcParams

# Use 'Agg' backend for non-GUI environments (optional but safe)
import matplotlib
matplotlib.use("Agg")

# Path to the Noto Sans Devanagari font (adjust if needed)
font_path = "/kaggle/input/notosans-devanagiri/static/NotoSansDevanagari-Regular.ttf"

if os.path.exists(font_path):
    font_manager.fontManager.addfont(font_path)
    dev_font = font_manager.FontProperties(fname=font_path)
    rcParams['font.family'] = dev_font.get_name()
    print(f"✅ Loaded font: {dev_font.get_name()}")
else:
    print("❌ Devanagari font not found. Falling back to default.")
    dev_font = None

# Test plot to render "भारत"
plt.figure(figsize=(6, 2))
plt.text(0.5, 0.5, "भारत", fontsize=30, ha='center', fontproperties=dev_font)
plt.title("Test: Devanagari Font Rendering", fontproperties=dev_font)
plt.axis('off')
plt.tight_layout()
plt.savefig("/kaggle/working/devanagari_test.png")
plt.show()


✅ Loaded font: Noto Sans Devanagari


In [18]:
plt.figure(figsize=(6, 2))
plt.text(0.5, 0.5, "भारत", fontsize=30, fontproperties=dev_font, ha='center')
plt.axis('off')
plt.title("Test: Devanagari Font Rendering", fontproperties=dev_font)
plt.show()


## **Implementation On Test Data**

In [10]:
import os
import random
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
from torch.utils.data import DataLoader
from IPython.display import display, HTML
from matplotlib import font_manager, rcParams
import wandb

# ============================ #
#   1. Font Setup for Hindi   #
# ============================ #
font_path = "/kaggle/input/notosans-devanagiri/static/NotoSansDevanagari-Regular.ttf"
assert os.path.exists(font_path), "Devanagari font not found."
devanagari_font = font_manager.FontProperties(fname=font_path)
font_manager.fontManager.addfont(font_path)
rcParams['font.family'] = devanagari_font.get_name()
print(f"Using Devanagari font: {devanagari_font.get_name()}")

# ============================ #
#   2. Device & W&B Init      #
# ============================ #
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
wandb.init(project="DL_ASSIGNMENT_3_RNN", name="test-inference-attention")

# ============================ #
#   3. Load Data & Vocab      #
# ============================ #
train_path = '/kaggle/input/dakshina/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv'
test_path = '/kaggle/input/dakshina/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv'
train_data = pd.read_csv(train_path, sep='\t', header=None, dtype=str).dropna()
test_data = pd.read_csv(test_path, sep='\t', header=None, dtype=str).dropna()

input_vocab = create_vocab(train_data, 1)
output_vocab = create_vocab(train_data, 0)
inv_input_vocab = {v: k for k, v in input_vocab.items()}
inv_output_vocab = {v: k for k, v in output_vocab.items()}

test_dataset = DakshinaDataset(test_data, input_vocab, output_vocab)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=pad_collate)

# ============================ #
#   4. Load Trained Model     #
# ============================ #
emb_dim = 32
hidden_dim = 256
num_layers = 3
dropout = 0.3
cell_type = 'LSTM'
beam_size = 3
sos_idx = output_vocab['<SOS>']
eos_idx = output_vocab['<EOS>']

attention = Attention(hidden_dim)
encoder = Attn_Encoder(len(input_vocab), emb_dim, hidden_dim, num_layers, cell_type, dropout)
decoder = Attn_Decoder(len(output_vocab), emb_dim, hidden_dim, num_layers, cell_type, dropout)
model = Attn_Seq2Seq(encoder, decoder, device, sos_idx, eos_idx).to(device)

model_path = '/kaggle/working/Attn_best_model.pt'
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# ============================ #
#   5. Inference & Viz        #
# ============================ #
correct_words, total_words = 0, 0
total_correct_chars, total_chars = 0, 0
predictions = []
viz_count, viz_samples = 0, 10

with torch.no_grad():
    for batch_idx, (src, tgt) in enumerate(test_loader):
        src, tgt = src.to(device), tgt.to(device)

        for i in range(src.size(0)):
            src_single = src[i].unsqueeze(0)
            tgt_single = tgt[i].unsqueeze(0)

            pred_seq, best_attn_list = model.predict(src_single, max_len=30, beam_size=beam_size)

            pred_indices = pred_seq.tolist()
            if eos_idx in pred_indices:
                pred_indices = pred_indices[:pred_indices.index(eos_idx)]
            pred_str = ''.join([inv_output_vocab.get(idx, '?') for idx in pred_indices])

            tgt_indices = tgt_single[0, 1:].tolist()
            if eos_idx in tgt_indices:
                tgt_indices = tgt_indices[:tgt_indices.index(eos_idx)]
            tgt_str = ''.join([inv_output_vocab.get(idx, '?') for idx in tgt_indices])

            input_indices = src_single[0].tolist()
            input_str = ''.join([inv_input_vocab.get(idx, '?') for idx in input_indices if idx not in [0, input_vocab['<EOS>']]])

            correct_word = pred_str == tgt_str
            correct_chars = sum(1 for p, t in zip(pred_str, tgt_str) if p == t)

            correct_words += correct_word
            total_words += 1
            total_correct_chars += correct_chars
            total_chars += len(tgt_str)

            predictions.append({
                'input': input_str,
                'target': tgt_str,
                'prediction': pred_str,
                'correct_word': correct_word,
                'correct_chars': correct_chars,
                'total_chars': len(tgt_str)
            })

            if viz_count < viz_samples:
                input_chars = [inv_input_vocab.get(idx, '?') for idx in input_indices if idx not in [0, input_vocab['<EOS>']]]
                pred_chars = [inv_output_vocab.get(idx, '?') for idx in pred_indices]
                attn_weights = [attn.cpu().numpy()[0] for attn in best_attn_list]

                fig, axes = plt.subplots(len(pred_chars), 1, figsize=(max(6, 0.7 * len(input_chars)), 1.5 * len(pred_chars) + 1))
                if len(pred_chars) == 1:
                    axes = [axes]

                fig.suptitle(f"Input: {input_str} -> Predicted: {pred_str}", fontsize=14, fontproperties=devanagari_font, y=1.05)

                for j, ax in enumerate(axes):
                    ax.set_title(f"Output char: {pred_chars[j]}", fontproperties=devanagari_font, fontsize=10)
                    for k, char in enumerate(input_chars):
                        weight = attn_weights[j][k]
                        ax.text(k * 0.7, 0, char, fontsize=14, ha='center', va='center',
                                bbox=dict(facecolor=plt.cm.Greens(weight), alpha=1, edgecolor='none', boxstyle='round,pad=0.2'),
                                fontproperties=devanagari_font)
                    ax.set_xlim(-0.5, len(input_chars) * 0.7)
                    ax.set_ylim(-0.5, 0.5)
                    ax.axis('off')

                plt.tight_layout(rect=[0, 0, 1, 0.95])
                viz_path = f'/kaggle/working/attent_@_sample_{viz_count}.png'
                plt.savefig(viz_path, dpi=150, bbox_inches='tight', pad_inches=0.1)
                plt.close(fig)
                wandb.log({f"attent_@_sample_{viz_count}": wandb.Image(viz_path)})
                viz_count += 1

# ============================ #
#   6. Combine All Images     #
# ============================ #
if viz_count > 0:
    grid_cols = 2
    grid_rows = (viz_count + grid_cols - 1) // grid_cols
    fig, axes = plt.subplots(grid_rows, grid_cols, figsize=(12, 1.2 * grid_rows * 5))

    for i in range(grid_rows * grid_cols):
        ax = axes.flat[i]
        if i < viz_count:
            img = mpimg.imread(f'/kaggle/working/attent_@_sample_{i}.png')
            ax.imshow(img)
            ax.set_title(f"Sample {i}", fontsize=12)
        ax.axis('off')

    plt.tight_layout()
    combined_path = '/kaggle/working/attent_@_samples_combined.png'
    plt.savefig(combined_path, dpi=150)
    plt.close(fig)
    wandb.log({"attent_@_samples_combined": wandb.Image(combined_path)})
    print(f"Saved combined attention image: {combined_path}")

# ============================ #
#   7. Log Accuracy + Table   #
# ============================ #
word_accuracy = 100 * correct_words / total_words
char_accuracy = 100 * total_correct_chars / total_chars
wandb.log({"test_word_accuracy": word_accuracy, "test_char_accuracy": char_accuracy})
print(f"\nWord Accuracy: {word_accuracy:.2f}%")
print(f"Char Accuracy: {char_accuracy:.2f}%")

# ============================ #
#   8. Save Predictions       #
# ============================ #
df_predictions = pd.DataFrame(predictions)
csv_path = '/kaggle/working/pred_attention.csv'
html_plain_path = '/kaggle/working/predictions_all_plain_attention.html'
html_colored_path = '/kaggle/working/predictions_sample_colored_attention.html'

df_predictions.to_csv(csv_path, index=False)
df_predictions.to_html(html_plain_path, index=False)

def highlight_row(row):
    return ['background-color: #d4edda;' if row['correct_word'] else 'background-color: #f8d7da;'] * len(row)

sample_df = pd.DataFrame(predictions[:viz_count])

styled_sample = sample_df.style.apply(highlight_row, axis=1)\
    .set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}])\
    .set_properties(**{'text-align': 'left', 'padding': '8px', 'font-size': '14px', 'border': '1px solid #ccc'})\
    .hide(axis='index')

with open(html_colored_path, 'w', encoding='utf-8') as f:
    f.write(f"<h3>Sample Predictions (Color-Coded)</h3>\n{styled_sample.to_html()}")

display(HTML("<h3>Sample Predictions (Color-Coded)</h3>"))
display(styled_sample)

# ============================ #
#   9. Log Artifacts to W&B   #
# ============================ #
artifact = wandb.Artifact('predictions_attention', type='predictions')
artifact.add_file(csv_path)
artifact.add_file(html_plain_path)
artifact.add_file(html_colored_path)
for i in range(viz_count):
    artifact.add_file(f'/kaggle/working/attent_@_sample_{i}.png')
if viz_count > 0:
    artifact.add_file(combined_path)
wandb.log_artifact(artifact)

wandb.log({
    "sample_predictions_table": wandb.Html(html_colored_path),
    "attention_samples_combined": wandb.Image(combined_path)
})

wandb.finish()


Using Devanagari font: Noto Sans Devanagari


Saved combined attention image: /kaggle/working/attent_@_samples_combined.png

Word Accuracy: 42.71%
Char Accuracy: 74.01%


input,target,prediction,correct_word,correct_chars,total_chars
ank,अंक,अंक,True,3,3
anka,अंक,अंका,False,3,3
ankit,अंकित,अंकित,True,5,5
anakon,अंकों,अनकों,False,4,5
ankhon,अंकों,अंखों,False,4,5
ankon,अंकों,अंकों,True,5,5
angkor,अंकोर,अंगकॉर,False,2,5
ankor,अंकोर,अंकोर,True,5,5
angaarak,अंगारक,अंगारक,True,6,6
angarak,अंगारक,अंगरक,False,3,6


test_char_accuracy,▁
test_word_accuracy,▁
test_char_accuracy,74.00953
test_word_accuracy,42.71435


## Heatmap Visulization

In [11]:
import wandb
import matplotlib.pyplot as plt
import numpy as np
import torch

wandb.init(project="DL_ASSIGNMENT_3_RNN", name="attention_heatmaps")

num_examples = 9
fig, axes = plt.subplots(3, 3, figsize=(18, 18))

for i, ax in enumerate(axes.flat[:num_examples]):
    src_tensor, tgt_tensor = test_dataset[i]
    src_tensor = src_tensor.unsqueeze(0).to(device)

    # Get prediction and attention weights
    pred_seq, attn_weights = model.predict(src_tensor, max_len=30, beam_size=beam_size)

    # Decode input tokens (exclude padding and EOS)
    src_indices = src_tensor[0].cpu().tolist()
    if input_vocab.get('<EOS>') in src_indices:
        src_indices = src_indices[:src_indices.index(input_vocab['<EOS>'])]
    input_tokens = [inv_input_vocab.get(idx, '?') for idx in src_indices if idx != 0]

    # Decode output tokens
    pred_indices = pred_seq.cpu().tolist()
    if output_vocab.get('<EOS>') in pred_indices:
        pred_indices = pred_indices[:pred_indices.index(output_vocab['<EOS>'])]
    output_tokens = [inv_output_vocab.get(idx, '?') for idx in pred_indices]

    # Stack attention weights list into 2D numpy array
    attn_array = torch.stack(attn_weights).squeeze(1).cpu().detach().numpy()

    # Slice to valid lengths (output tokens × input tokens)
    attn = attn_array[:len(output_tokens), :len(input_tokens)]

    # Plot attention heatmap on subplot
    im = ax.imshow(attn, cmap='viridis')

    ax.set_xticks(np.arange(len(input_tokens)))
    ax.set_xticklabels(input_tokens, rotation=45, fontsize=12)
    ax.set_yticks(np.arange(len(output_tokens)))
    ax.set_yticklabels(output_tokens, fontsize=12)

    ax.set_xlabel('Input (English)')
    ax.set_ylabel('Output (Hindi)')

    # Add input and output sequences as text above heatmap (wrapped for readability)
    input_sentence = " ".join(input_tokens)
    output_sentence = " ".join(output_tokens)
    ax.set_title(f'Sample {i+1}\nInput: {input_sentence}\nOutput: {output_sentence}', fontsize=10)

    # Log individual heatmap figure to wandb
    fig_single, ax_single = plt.subplots(figsize=(6, 6))
    im_single = ax_single.imshow(attn, cmap='viridis')
    ax_single.set_xticks(np.arange(len(input_tokens)))
    ax_single.set_xticklabels(input_tokens, rotation=45, fontsize=10)
    ax_single.set_yticks(np.arange(len(output_tokens)))
    ax_single.set_yticklabels(output_tokens, fontsize=10)
    ax_single.set_xlabel('Input (English)')
    ax_single.set_ylabel('Output (Hindi)')
    ax_single.set_title(f'Sample {i+1}\nInput: {input_sentence}\nOutput: {output_sentence}', fontsize=10)
    plt.tight_layout()

    wandb.log({f"attention_heatmap_sample_{i+1}": wandb.Image(fig_single)})
    plt.close(fig_single)

# Finalize the grid figure
plt.subplots_adjust(right=0.85, wspace=0.3, hspace=0.5)
fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.6, location='right')

# Log entire grid as one image to wandb
wandb.log({"attention_heatmaps_grid": wandb.Image(fig)})
plt.close(fig)


# Connectivity Visualization Of attention

In [13]:
# Animation
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
from matplotlib import font_manager, rcParams
from matplotlib import animation
import wandb

# ============================ #
#   1. Font Setup for Hindi   #
# ============================ #
font_path = "/kaggle/input/notosans-devanagiri/static/NotoSansDevanagari-Regular.ttf"
assert os.path.exists(font_path), "Devanagari font not found."
devanagari_font = font_manager.FontProperties(fname=font_path)
font_manager.fontManager.addfont(font_path)
rcParams['font.family'] = devanagari_font.get_name()
print(f"Using Devanagari font: {devanagari_font.get_name()}")

# ============================ #
#   2. Device & W&B Init      #
# ============================ #
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
wandb.init(project="DL_ASSIGNMENT_3_RNN", name="test-inference-attention-animated")

# ============================ #
#   3. Load Data & Vocab      #
# ============================ #
train_path = '/kaggle/input/dakshina/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv'
test_path = '/kaggle/input/dakshina/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv'
train_data = pd.read_csv(train_path, sep='\t', header=None, dtype=str).dropna()
test_data = pd.read_csv(test_path, sep='\t', header=None, dtype=str).dropna()

input_vocab = create_vocab(train_data, 1)
output_vocab = create_vocab(train_data, 0)
inv_input_vocab = {v: k for k, v in input_vocab.items()}
inv_output_vocab = {v: k for k, v in output_vocab.items()}

test_dataset = DakshinaDataset(test_data, input_vocab, output_vocab)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=pad_collate)

# ============================ #
#   4. Load Trained Model     #
# ============================ #
emb_dim = 32
hidden_dim = 256
num_layers = 3
dropout = 0.3
cell_type = 'LSTM'
beam_size = 3
sos_idx = output_vocab['<SOS>']
eos_idx = output_vocab['<EOS>']

attention = Attention(hidden_dim)
encoder = Attn_Encoder(len(input_vocab), emb_dim, hidden_dim, num_layers, cell_type, dropout)
decoder = Attn_Decoder(len(output_vocab), emb_dim, hidden_dim, num_layers, cell_type, dropout)
model = Attn_Seq2Seq(encoder, decoder, device, sos_idx, eos_idx).to(device)

model_path = '/kaggle/working/Attn_best_model.pt'
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# ============================ #
#  5. Attention Animation     #
# ============================ #
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.path import Path

def generate_attention_animation(input_chars, pred_chars, attn_weights, devanagari_font, save_path):
    fig, ax = plt.subplots(figsize=(max(6, 0.7 * len(input_chars)), 4))
    ax.set_xlim(-1, len(input_chars))
    ax.set_ylim(-2, 2)
    ax.axis('off')

    # Plot input characters on top
    input_positions = np.arange(len(input_chars))
    for i, ch in enumerate(input_chars):
        ax.text(i, 1.2, ch, fontsize=14, ha='center', fontproperties=devanagari_font)

    # Predicted char (changing per frame)
    pred_text = ax.text(len(input_chars)//2, -1.6, '', fontsize=16, ha='center', fontproperties=devanagari_font)

    # Create curves from bottom (pred char) to each input char
    curves = []
    for i in range(len(input_chars)):
        path_data = [
            (Path.MOVETO, (len(input_chars)//2, -1.2)),  # start: prediction char
            (Path.CURVE3, ((i + len(input_chars)//2) / 2, 0.0)),  # control point
            (Path.CURVE3, (i, 0.8))  # end: input char
        ]
        path = Path([p for _, p in path_data], [cmd for cmd, _ in path_data])
        patch = PathPatch(path, lw=0.5, alpha=0.0, color='green')
        ax.add_patch(patch)
        curves.append(patch)

    def init():
        for curve in curves:
            curve.set_alpha(0.0)
        pred_text.set_text('')
        return curves + [pred_text]

    def animate(t):
        for i, curve in enumerate(curves):
            alpha = attn_weights[t][i]
            curve.set_alpha(alpha)
            curve.set_linewidth(2.5 * alpha)
        pred_text.set_text(f"{pred_chars[t]}")
        return curves + [pred_text]

    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=len(pred_chars), interval=800, blit=True)
    anim.save(save_path, writer='pillow', dpi=100)
    plt.close(fig)


# ============================ #
#     6. Inference & Log      #
# ============================ #
predictions = []
viz_count, viz_samples = 0, 10
animation_info = []

with torch.no_grad():
    for batch_idx, (src, tgt) in enumerate(test_loader):
        src, tgt = src.to(device), tgt.to(device)
        for i in range(src.size(0)):
            src_single = src[i].unsqueeze(0)
            tgt_single = tgt[i].unsqueeze(0)

            pred_seq, best_attn_list = model.predict(src_single, max_len=30, beam_size=beam_size)

            pred_indices = pred_seq.tolist()
            if eos_idx in pred_indices:
                pred_indices = pred_indices[:pred_indices.index(eos_idx)]
            pred_str = ''.join([inv_output_vocab.get(idx, '?') for idx in pred_indices])

            tgt_indices = tgt_single[0, 1:].tolist()
            if eos_idx in tgt_indices:
                tgt_indices = tgt_indices[:tgt_indices.index(eos_idx)]
            tgt_str = ''.join([inv_output_vocab.get(idx, '?') for idx in tgt_indices])

            input_indices = src_single[0].tolist()
            input_str = ''.join([inv_input_vocab.get(idx, '?') for idx in input_indices if idx not in [0, input_vocab['<EOS>']]])

            predictions.append({
                'input': input_str,
                'target': tgt_str,
                'prediction': pred_str
            })

            if viz_count < viz_samples:
                input_chars = [inv_input_vocab.get(idx, '?') for idx in input_indices if idx not in [0, input_vocab['<EOS>']]]
                pred_chars = [inv_output_vocab.get(idx, '?') for idx in pred_indices]
                attn_weights = [attn.cpu().numpy()[0] for attn in best_attn_list]

                anim_path = f'/kaggle/working/attention_samples_{viz_count}.gif'
                generate_attention_animation(input_chars, pred_chars, attn_weights, devanagari_font, anim_path)

                wandb.save(anim_path)
                wandb.log({f"attention_samples_{viz_count}": wandb.Video(anim_path)})

                # Print or log words to stdout / console
                print(f"[{viz_count}] Input: {input_str} | Target: {tgt_str} | Prediction: {pred_str}")

                # Save animation info for HTML
                animation_info.append({
                    'gif_path': os.path.basename(anim_path),
                    'input': input_str,
                    'prediction': pred_str,
                    'target': tgt_str
                })

                viz_count += 1
        if viz_count >= viz_samples:
            break


# ============================ #
#   7. Save CSV Predictions   #
# ============================ #
pd.DataFrame(predictions).to_csv("/kaggle/working/predictions_with_attn.tsv", index=False)

# ============================ #
#   8. Create HTML Grid       #
# ============================ #
import base64

def embed_gif_base64(gif_path):
    with open(gif_path, "rb") as f:
        encoded = base64.b64encode(f.read()).decode('utf-8')
    return f"data:image/gif;base64,{encoded}"

html_str = '<h3>Attention Animations (Input → Prediction)</h3><table><tr>'

for i, anim in enumerate(animation_info):
    if i % 5 == 0 and i > 0:
        html_str += '</tr><tr>'  # new row after 5 columns

    escaped_input = anim['input'].replace('<', '&lt;').replace('>', '&gt;')
    escaped_pred = anim['prediction'].replace('<', '&lt;').replace('>', '&gt;')
    gif_path = f"/kaggle/working/{anim['gif_path']}"

    base64_img = embed_gif_base64(gif_path)

    html_str += f'''
    <td style="text-align:center;">
        <img src="{base64_img}" width="200"><br>
        <span style="font-size:14px;">{escaped_input} → {escaped_pred}</span>
    </td>
    '''

html_str += '</tr></table>'

# Save HTML to file
html_file_path = "/kaggle/working/attention_grids1.html"
with open(html_file_path, "w", encoding="utf-8") as f:
    f.write(html_str)

# Log to W&B
wandb.log({"attention_grids1": wandb.Html(html_str)})


wandb.finish()


Using Devanagari font: Noto Sans Devanagari


[0] Input: ank | Target: अंक | Prediction: अंक
[1] Input: anka | Target: अंक | Prediction: अंका
[2] Input: ankit | Target: अंकित | Prediction: अंकित
[3] Input: anakon | Target: अंकों | Prediction: अनकों
[4] Input: ankhon | Target: अंकों | Prediction: अंखों
[5] Input: ankon | Target: अंकों | Prediction: अंकों
[6] Input: angkor | Target: अंकोर | Prediction: अंगकॉर
[7] Input: ankor | Target: अंकोर | Prediction: अंकोर
[8] Input: angaarak | Target: अंगारक | Prediction: अंगारक
[9] Input: angarak | Target: अंगारक | Prediction: अंगरक
